### COLETA DE DADOS

In [1]:
# CARREGANDO DADOS
from ucimlrepo import fetch_ucirepo 

# importando dataset
dataset = fetch_ucirepo(id=848) # 61068 registros e 20 atributos
  
# coletando as informações
data_frame = dataset.data.original

### PRÉ-PROCESSAMENTO DE DADOS

In [2]:
# TRATANDO DADOS
import pandas
from sklearn.preprocessing import LabelEncoder

print("PRÉ TRATAMENTO: ", len(data_frame))

# removendo colunas com muitos nulos
tolerancia = len(data_frame) * 0.7
data_frame = data_frame.dropna(axis=1, thresh=tolerancia)

# removendo duplicados
print("VALORES DUPLICADOS: ", data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()

# convertendo colunas categóricas em valores inteiros
conversores = {}
for coluna in data_frame.columns:
    if (data_frame[coluna].dtype == type(object)):
        conversor = LabelEncoder()
        data_frame[coluna] = conversor.fit_transform(data_frame[coluna])
        conversores[coluna] = conversor

print("PÓS TRATAMENTO: ", len(data_frame))

PRÉ TRATAMENTO:  61069
VALORES DUPLICADOS:  146
PÓS TRATAMENTO:  60923


### DIVISÃO DE DADOS

In [3]:
print(data_frame.columns)

Index(['class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-color', 'stem-height',
       'stem-width', 'stem-color', 'has-ring', 'ring-type', 'habitat',
       'season'],
      dtype='object')


In [4]:
# DIVIDINDO O DATASET TRATADO
import numpy
from sklearn.model_selection import train_test_split

atributos = data_frame.drop(["class"], axis=1)
respostas = data_frame[["class"]]

a_treino, a_teste, r_treino, r_teste = train_test_split(atributos, respostas, test_size=0.3, random_state=42)

# convertendo de (N, 1) para (N,)
r_treino = numpy.ravel(r_treino)
r_teste = numpy.ravel(r_teste)

In [5]:
print("TAMANHO DO DATASET TRATADO: ", len(data_frame))
print("CONJUNTO DE TREINO: ", a_treino.shape, " e ", r_treino.shape)
print("CONJUNTO DE TESTE: ", a_teste.shape, " e ", r_teste.shape)

TAMANHO DO DATASET TRATADO:  60923
CONJUNTO DE TREINO:  (42646, 14)  e  (42646,)
CONJUNTO DE TESTE:  (18277, 14)  e  (18277,)


### TREINAMENTO E AVALIAÇÃO DO MODELO

In [6]:
# CRIANDO PARÂMETROS PARA OS MODELOS

lista_parametros = [
    {"id": "A", "n_estimators": 10, "criterion": "gini", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "B", "n_estimators": 100, "criterion": "gini", "max_depth": 10, "min_samples_split": 3, "min_samples_leaf": 10},
    {"id": "C", "n_estimators": 10, "criterion": "entropy", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "D", "n_estimators": 100, "criterion": "entropy", "max_depth": 10, "min_samples_split": 3, "min_samples_leaf": 10},
]

In [7]:
# APLICANDO MODELO
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# criando a lista de resultados finais
lista_resultados = []

for parametros in lista_parametros:
    # criando classificador
    classificador = RandomForestClassifier(
        n_estimators=parametros["n_estimators"], 
        criterion=parametros["criterion"], 
        max_depth=parametros["max_depth"], 
        min_samples_split=parametros["min_samples_split"], 
        min_samples_leaf=parametros["min_samples_leaf"], 
        random_state=42
    )

    # treinando o modelo
    classificador.fit(a_treino, r_treino)

    # prevendo respostas
    r_previsao = classificador.predict(a_teste)

    # calculando métricas
    acuracia = accuracy_score(r_teste, r_previsao)
    f1 = f1_score(r_teste, r_previsao, average="weighted")

    # salvando resultados
    lista_resultados.append([parametros["id"], acuracia, f1])

In [8]:
# EXIBINDO RESULTADOS
tabela_resultados = pandas.DataFrame(lista_resultados, columns=["Identificador", "Acurácia", "F1-score"])
print(tabela_resultados)

  Identificador  Acurácia  F1-score
0             A  0.998468  0.998468
1             B  0.990918  0.990916
2             C  0.998249  0.998249
3             D  0.985282  0.985285
